In [ ]:
!unzip "/content/drive/MyDrive/sneha zip/processed_B.zip" -d "/content/data"

Archive:  /content/drive/MyDrive/sneha zip/processed_B.zip
   creating: /content/data/processed_B/
   creating: /content/data/processed_B/density/
  inflating: /content/data/processed_B/density/IMG_1.npy  
  inflating: /content/data/processed_B/density/IMG_10.npy  
  inflating: /content/data/processed_B/density/IMG_100.npy  
  inflating: /content/data/processed_B/density/IMG_101.npy  
  inflating: /content/data/processed_B/density/IMG_102.npy  
  inflating: /content/data/processed_B/density/IMG_103.npy  
  inflating: /content/data/processed_B/density/IMG_104.npy  
  inflating: /content/data/processed_B/density/IMG_105.npy  
  inflating: /content/data/processed_B/density/IMG_106.npy  
  inflating: /content/data/processed_B/density/IMG_107.npy  
  inflating: /content/data/processed_B/density/IMG_108.npy  
  inflating: /content/data/processed_B/density/IMG_109.npy  
  inflating: /content/data/processed_B/density/IMG_11.npy  
  inflating: /content/data/processed_B/density/IMG_110.npy  
  i

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class CrowdDataset(Dataset):
    def __init__(self, img_paths, dens_paths, transform=None):
        self.img_paths = img_paths
        self.dens_paths = dens_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        dens = np.load(self.dens_paths[idx])  # (H,W)

        dens = torch.tensor(dens, dtype=torch.float32).unsqueeze(0)  # → shape: (1,H,W)

        if self.transform:
            img = self.transform(img)

        return img, dens

In [ ]:
import glob
from torchvision import transforms

images_path = "/content/data/processed_B/images/*.jpg"
density_path = "/content/data/processed_B/density/*.npy"

img_files = sorted(glob.glob(images_path))
den_files = sorted(glob.glob(density_path))

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = CrowdDataset(img_files, den_files, transform=transform)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import numpy as np

# ----------------------------------------------------
# 0. MOUNT GOOGLE DRIVE
# ----------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# Folder where you want to save your training files
drive_folder = "/content/drive/MyDrive/CSRNet_Training/"
os.makedirs(drive_folder, exist_ok=True)

checkpoint_path = os.path.join(drive_folder, "csrnet_checkpoint.pth")
stats_path = os.path.join(drive_folder, "training_stats.npz")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ----------------------------------------------------
# 1. MODEL (CSRNet)
# ----------------------------------------------------
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        self.frontend = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True)
        )

        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
        )

        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x


# ----------------------------------------------------
# 2. LOAD MODEL + RESUME CHECKPOINT
# ----------------------------------------------------
model = CSRNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

start_epoch = 1
loss_history = []
gt_history = []
pred_history = []

# If checkpoint exists → resume
if os.path.exists(checkpoint_path):
    print("🔄 Resuming training from checkpoint in Drive...")
    ckpt = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    start_epoch = ckpt["epoch"] + 1

    if os.path.exists(stats_path):
        data = np.load(stats_path, allow_pickle=True)
        loss_history = list(data["loss_history"])
        gt_history = list(data["gt_history"])
        pred_history = list(data["pred_history"])
else:
    print("🆕 Starting training from scratch!")


# ----------------------------------------------------
# 3. TRAIN LOOP
# ----------------------------------------------------
num_epochs = 100
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    total_gt = 0.0
    total_pred = 0.0

    pbar = tqdm(train_loader)

    for img, dens in pbar:
        img = img.to(device)
        dens = dens.to(device)

        optimizer.zero_grad()
        pred = model(img)

        # Resize to match GT size
        if pred.shape != dens.shape:
            pred = F.interpolate(pred, size=dens.shape[2:], mode="bilinear", align_corners=False)

        loss = F.mse_loss(pred, dens)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        epoch_loss += batch_loss

        gt_count = dens.sum().item()
        pred_count = pred.sum().item()

        total_gt += gt_count
        total_pred += pred_count

        pbar.set_description(
            f"Epoch {epoch}/{num_epochs} | loss={batch_loss:.6f} | gt={gt_count:.1f} | pred={pred_count:.1f}"
        )

    # Save epoch stats
    loss_history.append(epoch_loss / len(train_loader))
    gt_history.append(total_gt)
    pred_history.append(total_pred)

    print(f"\n--- Epoch {epoch}/{num_epochs} | AvgLoss={loss_history[-1]:.6f} "
          f"| GT={total_gt:.2f} | Pred={total_pred:.2f} | Diff={abs(total_gt-total_pred):.2f} ---\n")

    # Save checkpoint to Google Drive
    torch.save({
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict()
    }, checkpoint_path)

    # Save stats
    np.savez(stats_path,
             loss_history=np.array(loss_history, dtype=float),
             gt_history=np.array(gt_history, dtype=float),
             pred_history=np.array(pred_history, dtype=float))

    print("💾 Saved checkpoint + stats to Google Drive!\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🆕 Starting training from scratch!


Epoch 1/100 | loss=0.000221 | gt=38.0 | pred=116.4: 100%|██████████| 400/400 [02:48<00:00,  2.37it/s]



--- Epoch 1/100 | AvgLoss=0.002365 | GT=49151.00 | Pred=107977.85 | Diff=58826.85 ---

💾 Saved checkpoint + stats to Google Drive!



Epoch 2/100 | loss=0.000951 | gt=238.0 | pred=162.8: 100%|██████████| 400/400 [02:50<00:00,  2.34it/s]



--- Epoch 2/100 | AvgLoss=0.001228 | GT=49151.00 | Pred=48250.74 | Diff=900.26 ---

💾 Saved checkpoint + stats to Google Drive!



...


--- Epoch 98/100 | AvgLoss=0.000304 | GT=49151.00 | Pred=48864.40 | Diff=286.60 ---

💾 Saved checkpoint + stats to Google Drive!



Epoch 99/100 | loss=0.000715 | gt=340.0 | pred=293.6: 100%|██████████| 400/400 [02:51<00:00,  2.34it/s]



--- Epoch 99/100 | AvgLoss=0.000284 | GT=49151.00 | Pred=49474.18 | Diff=323.18 ---

💾 Saved checkpoint + stats to Google Drive!



Epoch 100/100 | loss=0.000317 | gt=238.0 | pred=186.0: 100%|██████████| 400/400 [02:51<00:00,  2.34it/s]



--- Epoch 100/100 | AvgLoss=0.000281 | GT=49151.00 | Pred=49373.11 | Diff=222.10 ---

💾 Saved checkpoint + stats to Google Drive!



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import numpy as np
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        self.frontend = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True)
        )

        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
        )

        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x


In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm
from math import sqrt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# =========================================================
# 1. TEST DATASET CLASS
# =========================================================
class TestDataset(Dataset):
    def __init__(self, img_dir, den_dir, transform=None):
        self.img_dir = img_dir
        self.den_dir = den_dir
        self.transform = transform

        self.img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(".jpg") or f.endswith(".png")])
        self.den_files = sorted([f for f in os.listdir(den_dir) if f.endswith(".npy")])

        assert len(self.img_files) == len(self.den_files), "❌ Images and density maps count mismatch"

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        den_path = os.path.join(self.den_dir, self.den_files[idx])

        # Load image
        img = Image.open(img_path).convert("RGB")
        img = np.array(img, dtype=np.float32) / 255.0
        img = torch.tensor(img).permute(2, 0, 1)  # HWC → CHW

        # Load density map
        den = np.load(den_path)  # (H, W)
        den = torch.tensor(den, dtype=torch.float32).unsqueeze(0)

        return img, den


# =========================================================
# 2. SET YOUR PATHS HERE
# =========================================================
import os

test_img_dir = "processed_B/images"
test_den_dir = "processed_B/density"
# test_img_dir = r"C:\Users\sneha\.cache\kagglehub\datasets\tthien\shanghaitech\versions\1\ShanghaiTech\part_A\train_data\images"
# test_den_dir = r"C:\Users\sneha\.cache\kagglehub\datasets\tthien\shanghaitech\versions\1\ShanghaiTech\part_A\train_data\ground-truth"


test_dataset = TestDataset(test_img_dir, test_den_dir)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"Loaded {len(test_dataset)} test images ✔")


# =========================================================
# 3. LOAD MODEL & CHECKPOINT
# =========================================================
model = CSRNet().to(device)

checkpoint_path = "sneha/csrnet_checkpoint.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)

model.load_state_dict(checkpoint["model"])
model.eval()

print("✅ Model loaded successfully!")


# =========================================================
# 4. POST PROCESSING
# =========================================================
def smooth(pred):
    x = pred[0, 0].cpu().numpy()
    return cv2.GaussianBlur(x, (5, 5), 0)


def clip(pred):
    cap = np.percentile(pred, 99)
    return np.clip(pred, 0, cap)


def correction(count):
    return count * 0.92


# =========================================================
# 5. EVALUATION LOOP
# =========================================================
mae = 0
mse = 0

print("\nStarting evaluation...")

for img, gt_den in tqdm(test_loader):
    img = img.to(device)
    gt_den = gt_den.to(device)

    with torch.no_grad():
        pred = model(img)
        pred = F.interpolate(pred, size=gt_den.shape[2:], mode='bilinear', align_corners=False)

    raw = pred.sum().item()

    # Post-processing
    sm = smooth(pred)
    cp = clip(sm)
    final_count = correction(cp.sum())

    gt = gt_den.sum().item()

    mae += abs(final_count - gt)
    mse += (final_count - gt) ** 2


MAE = mae / len(test_loader)
MSE = mse / len(test_loader)
RMSE = sqrt(MSE)

print("\n==============================")
print("📊 FINAL TEST RESULTS")
print("==============================")
print(f"⭕ MAE  = {MAE:.2f}")
print(f"📘 MSE  = {MSE:.2f}")
print(f"📏 RMSE = {RMSE:.2f}")
print("==============================")


Loaded 400 test images ✔
✅ Model loaded successfully!

Starting evaluation...


100%|██████████| 400/400 [57:55<00:00,  8.69s/it] 


📊 FINAL TEST RESULTS
⭕ MAE  = 49.39
📘 MSE  = 4093.74
📏 RMSE = 63.98
